# Velocity and Acceleration in Pandas Tutorial

This notebook explains how to create velocity and acceleration features in `pandas`.

This notebook will use gold and silver price data from `rdatasets` for this tutorial

### Packages

The documentation for each package used in this tutorial is linked below:
* [pandas](https://pandas.pydata.org/docs/)
* [statsmodels](https://www.statsmodels.org/stable/index.html)
    * [statsmodels.api](https://www.statsmodels.org/stable/api.html#statsmodels-api)

In [1]:
import statsmodels.api as sm
import pandas as pd

## Create initial dataset

The data is from `rdatasets` imported using the Python package `statsmodels`.

In [2]:
df = sm.datasets.get_rdataset('GoldSilver', 'AER').data.reset_index().rename(columns={'index': 'date'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9132 entries, 0 to 9131
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    9132 non-null   object 
 1   gold    9132 non-null   float64
 2   silver  9132 non-null   float64
dtypes: float64(2), object(1)
memory usage: 214.2+ KB


In [3]:
df['date'] = pd.to_datetime(df.date)

## Time series aggregation

The `pandas` function `resample` can be used to create aggregations on specified windows.  Here, a weekly aggregate of the daily gold and silver price data will be created.  

First, a datetime index needs to be created from the **date** column.

In [4]:
df.set_index('date', inplace=True)

The parameter **origin** is used to specify the day of week to start the aggregation on.  January 1, 1978 is a Sunday, so each of these aggreagtions will start on Sunday and aggregate over the subsequent week.

In [5]:
weekly_resample = df.resample('7D', origin=pd.to_datetime('1978-01-01'))
aggregated_df = weekly_resample.agg(['min', 'mean', 'max', 'std'])
aggregated_df.columns = ['_'.join(col).strip() + '_week' for col in aggregated_df.columns.values]

### Velocity

Calculate the velocity of the mean weekly gold price as the difference between means divided by the number of days between.

In [8]:
aggregated_df['gold_velocity_1wk'] = (aggregated_df['gold_mean_week'] - aggregated_df['gold_mean_week'].shift(1))/7
aggregated_df['gold_velocity_4wk'] = (aggregated_df['gold_mean_week'] - aggregated_df['gold_mean_week'].shift(4))/28
aggregated_df.head(20)

,gold_min_week,gold_mean_week,gold_max_week,gold_std_week,silver_min_week,silver_mean_week,silver_max_week,silver_std_week,gold_velocity,gold_velocity_1wk,gold_velocity_4wk
date,,,,,,,,,,,
1977-12-25,100.00,100.000,100.00,NaN,223.42,223.420,223.42,NaN,NaN,NaN,NaN
1978-01-01,100.00,100.000,100.00,0.000000,223.42,226.604,229.84,2.596657,0.000000,0.000000,NaN
1978-01-08,100.88,101.586,102.62,0.796825,227.89,230.050,232.55,1.846334,0.226571,0.226571,NaN
1978-01-15,100.87,101.832,103.09,0.796976,227.96,229.686,232.49,1.755130,0.035143,0.035143,NaN
1978-01-22,102.49,103.306,103.62,0.475742,230.37,232.798,234.50,1.777630,0.210571,0.210571,0.118071
1978-01-29,101.96,102.578,103.49,0.636530,226.91,229.116,231.61,1.936835,-0.104000,-0.104000,0.092071
1978-02-05,101.90,102.572,103.15,0.505193,224.98,227.506,229.90,1.875361,-0.000857,-0.000857,0.035214
1978-02-12,103.92,104.936,105.87,0.850312,228.16,231.506,234.01,2.412370,0.337714,0.337714,0.110857
1978-02-19,105.14,105.956,107.23,0.822028,229.81,232.392,234.55,1.784550,0.145714,0.145714,0.094643


### Accelerations

Calculate the accelerations of the mean weekly gold price as the difference between velocities divided by the number of days between.

In [9]:
aggregated_df['gold_acceleration_1wk'] = (aggregated_df['gold_velocity_1wk'] - aggregated_df['gold_velocity_1wk'].shift(1))/7
aggregated_df['gold_acceleration_4wk'] = (aggregated_df['gold_velocity_4wk'] - aggregated_df['gold_velocity_4wk'].shift(4))/28
aggregated_df.head(20)

,gold_min_week,gold_mean_week,gold_max_week,gold_std_week,silver_min_week,silver_mean_week,silver_max_week,silver_std_week,gold_velocity,gold_velocity_1wk,gold_velocity_4wk,gold_acceleration_1wk,gold_acceleration_4wk
date,,,,,,,,,,,,,
1977-12-25,100.00,100.000,100.00,NaN,223.42,223.420,223.42,NaN,NaN,NaN,NaN,NaN,NaN
1978-01-01,100.00,100.000,100.00,0.000000,223.42,226.604,229.84,2.596657,0.000000,0.000000,NaN,NaN,NaN
1978-01-08,100.88,101.586,102.62,0.796825,227.89,230.050,232.55,1.846334,0.226571,0.226571,NaN,0.032367,NaN
1978-01-15,100.87,101.832,103.09,0.796976,227.96,229.686,232.49,1.755130,0.035143,0.035143,NaN,-0.027347,NaN
1978-01-22,102.49,103.306,103.62,0.475742,230.37,232.798,234.50,1.777630,0.210571,0.210571,0.118071,0.025061,NaN
1978-01-29,101.96,102.578,103.49,0.636530,226.91,229.116,231.61,1.936835,-0.104000,-0.104000,0.092071,-0.044939,NaN
1978-02-05,101.90,102.572,103.15,0.505193,224.98,227.506,229.90,1.875361,-0.000857,-0.000857,0.035214,0.014735,NaN
1978-02-12,103.92,104.936,105.87,0.850312,228.16,231.506,234.01,2.412370,0.337714,0.337714,0.110857,0.048367,NaN
1978-02-19,105.14,105.956,107.23,0.822028,229.81,232.392,234.55,1.784550,0.145714,0.145714,0.094643,-0.027429,-0.000837
